# Составление фондового индекса

###### Импорт необходимых библиотек

In [7]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA

###### Импорт датасета 

In [9]:
df = pd.read_csv('close_prices.csv', index_col='date')
df.head(5)

,AXP,BA,CAT,CSCO,CVX,DD,DIS,GE,GS,HD,...,PFE,PG,T,TRV,UNH,UTX,V,VZ,WMT,XOM
date,,,,,,,,,,,,,,,,,,,,,
2013-09-23,76.440002,117.510002,85.029999,24.270000,125.519997,59.409999,64.750000,24.280001,165.250000,75.910004,...,28.799999,79.279999,34.220001,86.379997,71.820000,109.419998,196.240005,47.980000,76.419998,87.750000
2013-09-24,76.070000,119.000000,85.110001,24.139999,124.489998,59.319997,64.320000,24.320000,162.970001,76.040001,...,28.709999,78.620003,34.090000,85.870003,72.320000,110.000000,193.339996,47.270000,75.750000,87.360001
2013-09-25,75.989998,118.510002,84.500000,24.430000,124.070000,59.319997,64.449997,24.230000,162.309998,75.519997,...,28.490000,77.720001,34.049999,85.980003,71.980003,109.260002,191.559998,46.950001,74.650002,87.139999
2013-09-26,76.320000,119.379997,84.199997,23.770000,123.489998,59.509996,65.239998,24.250000,162.289993,76.070000,...,28.520000,78.050003,34.230000,85.830002,72.160004,109.660004,193.559998,47.669998,74.620003,87.070000
2013-09-27,75.889999,118.739998,83.800003,23.330000,122.639999,59.009995,65.190002,24.049999,159.850006,75.959999,...,28.879999,77.209999,33.980000,85.410004,71.989998,109.360001,193.050003,47.000000,74.360001,86.900002


###### Использование метода главных компонент с датасетом

In [24]:
# Создание и обучение метода главных компонент на датасете
PCA_prices = PCA(n_components=10)
PCA_prices.fit(df)

PCA(n_components=10)

In [68]:
# Подсчет количества компонент, объясняющих 90% дисперсии признаков исходного датасета
n_components = 0
explained_variance = 0
for i in PCA_prices.explained_variance_ratio_:
    n_components += 1
    explained_variance += i
    if round(explained_variance * 100) > 90:
        break
print(f'Дисперсия в 90% объясняется {n_components} компонентами')
print(f'Объясненная дисперсия равна {round(explained_variance * 100, 2)}%')

Дисперсия в 90% объясняется 4 компонентами
Объясненная дисперсия равна 92.77%


In [27]:
# Преобразование исходного датасета с методом главных компонент
df_PCA = PCA_prices.transform(df)

In [32]:
df_PCA = pd.DataFrame(df_PCA)
df_PCA.head(5)

,0,1,2,3,4,5,6,7,8,9
0,-50.902404,17.631677,7.736021,2.596010,2.377569,4.244055,12.883088,3.556570,-5.821974,-1.726040
1,-52.846909,19.146907,7.272546,4.618575,0.102182,3.935519,11.062930,3.432599,-5.633181,-2.012232
2,-54.614439,19.927358,8.109958,4.920347,-0.421454,4.555335,10.119224,4.175203,-5.388287,-2.752699
3,-52.600566,19.941571,6.948113,4.806243,0.319779,5.065505,9.849605,3.266579,-5.974574,-1.922746
4,-52.370123,22.289064,8.410868,3.833372,-2.464882,4.359897,7.988691,3.452709,-5.745204,-1.875131


In [43]:
# Отбор значений первой компоненты
first_component = np.expand_dims(df_PCA[0], axis=1)

###### Импорт второго датасета (Индекс Доу-Джонса) и расчет корреляции с первой компонентой

In [39]:
dou_index = pd.read_csv('djia_index.csv', index_col='date')
dou_index.head(5)

,^DJI
date,
2013-09-23,15401.379883
2013-09-24,15334.589844
2013-09-25,15273.259766
2013-09-26,15328.299805
2013-09-27,15258.240234


In [66]:
# Расчет корреляции между индексом и первой компонентой
correlation = np.corrcoef(first_component.T, dou_index.T)[0, 1]
print(f'Корреляция Пирсона между индексом Доу-Джонса и первой компонентой равна {round(correlation, 2)}')

Корреляция между индексом Доу-Джонса и первой компонентой равна 0.91


In [63]:
most_valued_company = df.columns[np.argmax(PCA_prices.components_[0])]
print(f'Наибольший вес в первой компоненте имеет компания "{most_valued_company}"')

Наибольший вес в первой компоненте имеет компания "V"
